In [1]:
import torch
import torch.nn as nn

# Define as dimensões da imagem como referência
IMG_HEIGHT = 40
IMG_WIDTH = 200
IMG_CHANNELS = 4  # BGRA do Webots

class CNNNavigationModel(nn.Module):
    """
    Define a arquitetura de um modelo CNN robusto para navegação usando dados de câmera e LiDAR.

    Este modelo aprimorado calcula dinamicamente o tamanho da entrada para as camadas
    totalmente conectadas, tornando-o robusto a mudanças nas dimensões da imagem de entrada
    ou na arquitetura da CNN. Ele também refatora a ramificação da câmera para maior clareza
    e inclui inicialização de pesos Kaiming.
    """
    def __init__(self, lidar_shape_in, img_shape=(IMG_CHANNELS, IMG_HEIGHT, IMG_WIDTH)):
        """
        Inicializa o modelo CNN.

        Args:
            lidar_shape_in (int): O número de características de entrada para a ramificação LiDAR.
            img_shape (tuple): A forma da imagem de entrada (C, H, W).
        """
        super(CNNNavigationModel, self).__init__()

        # --- Ramificação da Câmera ---_
        # Definimos a parte convolucional primeiro para calcular dinamicamente seu tamanho de saída.
        camera_conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=img_shape[0], out_channels=32, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
        )

        # **MELHORIA 1: Cálculo dinâmico do tamanho das características**
        # Criamos um tensor "dummy" e o passamos pelas camadas convolucionais para descobrir o tamanho achatado.
        with torch.no_grad():
            dummy_input = torch.zeros(1, *img_shape)
            flattened_cam_size = camera_conv_layers(dummy_input).shape[1]
            print(f"Tamanho da característica da câmera achatada calculado dinamicamente: {flattened_cam_size}")


        # **MELHORIA 2: Clareza arquitetural**
        # Combinamos todo o pipeline de processamento da câmera em um único módulo sequencial.
        self.camera_branch = nn.Sequential(
            camera_conv_layers,
            nn.Linear(flattened_cam_size, 64),
            nn.ReLU()
        )

        # --- Ramificação do LiDAR ---_
        self.lidar_branch = nn.Sequential(
            nn.Linear(lidar_shape_in, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU()
        )

        # --- Cabeça Combinada ---_
        # Esta parte permanece conceitualmente a mesma.
        self.combined_head = nn.Sequential(
            nn.Linear(64 + 64, 128),  # 64 da câmera + 64 do LiDAR
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2),  # Saída de 2 valores (dist, angulo)
            nn.Tanh()          # Escala a saída para [-1, 1] para alvos normalizados
        )

        # **MELHORIA 3: Inicialização de Pesos**
        self.apply(self._init_weights)

    def _init_weights(self, module):
        """
        Aplica a inicialização Kaiming He às camadas Conv2d e Linear.
        """
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            nn.init.kaiming_normal_(module.weight, mode='fan_in', nonlinearity='relu')
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)

    def forward(self, cam_input, lidar_input):
        """
        Define a passagem para a frente (forward pass) do modelo.

        Args:
            cam_input (torch.Tensor): O tensor de entrada da câmera.
                                      Forma: (N, C, H, W)
            lidar_input (torch.Tensor): O tensor de entrada do LiDAR.
                                        Forma: (N, lidar_shape_in)

        Returns:
            torch.Tensor: O tensor de saída do modelo. Forma: (N, 2)
        """
        cam_features = self.camera_branch(cam_input)
        lidar_features = self.lidar_branch(lidar_input)
        combined_features = torch.cat((cam_features, lidar_features), dim=1)
        output = self.combined_head(combined_features)
        return output

In [3]:
import sys
import os
import h5py
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold
import matplotlib
matplotlib.use('Agg')  # Define o backend não-interativo explicitamente

# --- Configuration ---_
SCRIPT_DIR = Path.cwd()
DATASET_PATH = SCRIPT_DIR / 'cnn_dataset.h5'
# Diretório para salvar os checkpoints
CHECKPOINT_DIR = SCRIPT_DIR / 'checkpoints'
PLOT_SAVE_PATH = SCRIPT_DIR / 'training_history.png'

# Cria o diretório de checkpoints se ele não existir
CHECKPOINT_DIR.mkdir(exist_ok=True)

# Image dimensions
IMG_HEIGHT = 40
IMG_WIDTH = 200
IMG_CHANNELS = 4  # BGRA from Webots

# Training parameters
EPOCHS = 40
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
N_SPLITS = 5  # Número de folds para K-Fold

# --- Device Configuration ---_
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(path: Path, max_samples=None):
    """
    Carrega o dataset de um único arquivo HDF5.
    """
    if not path.exists() or not path.is_file():
        print(f"Error: HDF5 file not found at '{path}'")
        raise FileNotFoundError

    with h5py.File(path, 'r') as hf:
        if 'camera_image' not in hf:
            print(f"Error: 'camera_image' dataset not found in '{path}'")
            raise ValueError

        print(f"Loading samples from '{path}'...")
        X_cam = np.array(hf['camera_image'], dtype=np.float32)
        X_lidar = np.array(hf['lidar_data'], dtype=np.float32)
        dist = np.array(hf['dist'], dtype=np.float32)
        angle = np.array(hf['angle'], dtype=np.float32)
        y = np.stack((dist, angle), axis=1)

        if max_samples:
            X_cam = X_cam[:max_samples]
            X_lidar = X_lidar[:max_samples]
            y = y[:max_samples]

    # Normaliza imagens
    X_cam /= 255.0

    # --- NORMALIZE TARGETS (y) ---_
    y[:, 0] /= 3.14  # Normalize distance
    y[:, 1] /= np.pi   # Normalize angle
    y = np.clip(y, -1.0, 1.0)

    # Normaliza dados do LiDAR
    if X_lidar.size > 0:
        max_lidar_val = np.max(X_lidar[np.isfinite(X_lidar)])
        if max_lidar_val > 0:
            X_lidar[np.isinf(X_lidar)] = max_lidar_val
            X_lidar /= max_lidar_val

    # Permuta os eixos da imagem para o formato do PyTorch (N, C, H, W)
    X_cam = np.transpose(X_cam, (0, 3, 1, 2))

    return (X_cam, X_lidar), y

Using device: cuda


In [2]:
def plot_history(history):
    """Plota o histórico de treinamento e validação e salva em um arquivo."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    ax1.plot(history['loss'], label='Training Loss')
    ax1.plot(history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_ylabel('Loss (MSE)')
    ax1.set_xlabel('Epoch')
    ax1.legend()
    ax2.plot(history['mae'], label='Training MAE')
    ax2.plot(history['val_mae'], label='Validation MAE')
    ax2.set_title('Model MAE')
    ax2.set_ylabel('MAE')
    ax2.set_xlabel('Epoch')
    ax2.legend()
    plt.tight_layout()
    plt.savefig(PLOT_SAVE_PATH)  # Salva a figura em um arquivo
    plt.close(fig)  # Fecha a figura para liberar memória
    print(f"Training plot saved to '{PLOT_SAVE_PATH}'")
    plt.close(fig)  # Fecha a figura para liberar memória

In [ ]:
(X_cam, X_lidar), y = load_data(DATASET_PATH)


Loading samples from '/home/dino/Documents/ia/controllers/cnn/cnn_dataset.h5'...


In [30]:
(X_cam_test, X_lidar_test), y_test = load_data(Path('/home/dino/Documents/ia/controllers/cnn/test.h5'))  # /home/dino/Documents/ia/controllers/cnn/test.h5

Loading samples from '/home/dino/Documents/ia/controllers/cnn/test.h5'...


Max distance: -0.189000204205513
Max angle: -0.3189796805381775


In [28]:

if X_cam is None or X_cam.shape[0] == 0:
    print("Exiting: No data loaded.")
else:
    print(f"Data loaded. Shapes: Cam={X_cam.shape}, Lidar={X_lidar.shape}, Target={y.shape}")

    kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
    fold_histories = []

    best_overall_val_loss = float('inf')
    best_overall_checkpoint_path = None

    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_cam)):
        print(f"\n--- Starting Fold {fold+1}/{N_SPLITS} ---")

        # Define o caminho do checkpoint para este fold específico
        checkpoint_path = CHECKPOINT_DIR / f'checkpoint_fold_{fold+1}.pth'

        # Split data for this fold
        X_cam_train, y_train = X_cam[train_idx], y[train_idx]
        X_lidar_train = X_lidar[train_idx]

        X_cam_val, y_val = X_cam[val_idx], y[val_idx]
        X_lidar_val = X_lidar[val_idx]

        # Converte dados para tensores PyTorch
        train_dataset = TensorDataset(torch.tensor(
            X_cam_train), torch.tensor(X_lidar_train), torch.tensor(y_train))
        train_loader = DataLoader(
            train_dataset, batch_size=BATCH_SIZE, shuffle=True)

        val_dataset = TensorDataset(torch.tensor(
            X_cam_val), torch.tensor(X_lidar_val), torch.tensor(y_val))
        val_loader = DataLoader(
            val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # Instancia o modelo e o otimizador
        model = CNNNavigationModel(lidar_shape_in=X_lidar.shape[1]).to(device)
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        # Inicializa variáveis para o loop de treino e checkpoint
        start_epoch = 0
        best_val_loss = float('inf')
        history = {'loss': [], 'val_loss': [], 'mae': [],
                   'val_mae': [], "best_val_mae": 0.0}

        # --- LÓGICA PARA CARREGAR CHECKPOINT ---_
        if checkpoint_path.exists():
            print(f"Resuming training from checkpoint: {checkpoint_path}")
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint['epoch'] + 1
            best_val_loss = checkpoint['best_val_loss']
            history = checkpoint['history']
            print(
                f"Resumed from epoch {start_epoch}. Best validation loss so far: {best_val_loss:.4f}")
        else:
            print(
                f"No checkpoint found for fold {fold+1}. Starting from scratch.")

        # Define as funções de perda
        criterion = nn.MSELoss()
        mae_criterion = nn.L1Loss()

        print(
            f"\n--- Starting Model Training for Fold {fold+1} from Epoch {start_epoch+1} ---")
        for epoch in range(start_epoch, EPOCHS):
            # --- Fase de Treinamento ---_
            model.train()
            running_loss, running_mae = 0.0, 0.0
            progress_bar = tqdm(
                train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}/{EPOCHS} [T]", leave=False)
            for cam_batch, lidar_batch, target_batch in progress_bar:
                cam_batch, lidar_batch, target_batch = cam_batch.to(
                    device), lidar_batch.to(device), target_batch.to(device)
                optimizer.zero_grad()
                outputs = model(cam_batch, lidar_batch)
                loss = criterion(outputs, target_batch)
                mae = mae_criterion(outputs, target_batch)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                running_mae += mae.item()
                progress_bar.set_postfix(loss=loss.item(), mae=mae.item())

            epoch_loss = running_loss / len(train_loader)
            epoch_mae = running_mae / len(train_loader)
            history['loss'].append(epoch_loss)
            history['mae'].append(epoch_mae)

            # --- Fase de Validação ---_
            model.eval()
            val_loss, val_mae = 0.0, 0.0
            with torch.no_grad():
                for cam_batch, lidar_batch, target_batch in val_loader:
                    cam_batch, lidar_batch, target_batch = cam_batch.to(
                        device), lidar_batch.to(device), target_batch.to(device)
                    outputs = model(cam_batch, lidar_batch)
                    val_loss += criterion(outputs, target_batch).item()
                    val_mae += mae_criterion(outputs, target_batch).item()

            epoch_val_loss = val_loss / len(val_loader)
            epoch_val_mae = val_mae / len(val_loader)
            history['val_loss'].append(epoch_val_loss)
            history['val_mae'].append(epoch_val_mae)

            print(
                f"Fold {fold+1}/{N_SPLITS} Epoch {epoch+1}/{EPOCHS} - "
                f"Loss: {epoch_loss:.4f}, MAE: {epoch_mae:.4f} - "
                f"Val Loss: {epoch_val_loss:.4f}, Val MAE: {epoch_val_mae:.4f}"
            )

            # --- LÓGICA PARA SALVAR CHECKPOINT ---_
            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                print(
                    f"  -> New best validation loss: {best_val_loss:.4f}. Saving checkpoint...")
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_val_loss': best_val_loss,
                    'best_val_mae': epoch_val_mae,
                    'history': history
                }, checkpoint_path)

                history['best_val_mae'] = epoch_val_mae
        fold_histories.append(history)

        # Update best overall checkpoint if this fold performed better
        if checkpoint_path.exists():
            checkpoint = torch.load(checkpoint_path, map_location=device)
            current_fold_best_val_loss = checkpoint['best_val_loss']
            if current_fold_best_val_loss < best_overall_val_loss:
                best_overall_val_loss = current_fold_best_val_loss
                best_overall_checkpoint_path = checkpoint_path

    # Agrega e plota os resultados de todos os folds
    if fold_histories:
        avg_history = {
            'loss': np.mean([h['loss'] for h in fold_histories], axis=0).tolist(),
            'val_loss': np.mean([h['val_loss'] for h in fold_histories], axis=0).tolist(),
            'mae': np.mean([h['mae'] for h in fold_histories], axis=0).tolist(),
            'val_mae': np.mean([h['val_mae'] for h in fold_histories], axis=0).tolist(),
            'best_val_mae': np.mean([h['best_val_mae'] for h in fold_histories]).tolist()
        }
        plot_history(avg_history)

    print("\nTraining complete.")

Data loaded. Shapes: Cam=(7493, 4, 40, 200), Lidar=(7493, 20), Target=(7493, 2)

--- Starting Fold 1/5 ---
Tamanho da característica da câmera achatada calculado dinamicamente: 24576
Resuming training from checkpoint: /home/dino/Documents/ia/controllers/cnn/checkpoints/checkpoint_fold_1.pth
Resumed from epoch 40. Best validation loss so far: 0.0031

--- Starting Model Training for Fold 1 from Epoch 41 ---

--- Starting Fold 2/5 ---
Tamanho da característica da câmera achatada calculado dinamicamente: 24576
Resuming training from checkpoint: /home/dino/Documents/ia/controllers/cnn/checkpoints/checkpoint_fold_2.pth
Resumed from epoch 39. Best validation loss so far: 0.0022

--- Starting Model Training for Fold 2 from Epoch 40 ---


Fold 2/5 Epoch 40/40 - Loss: 0.0032, MAE: 0.0372 - Val Loss: 0.0026, Val MAE: 0.0288

--- Starting Fold 3/5 ---
Tamanho da característica da câmera achatada calculado dinamicamente: 24576
Resuming training from checkpoint: /home/dino/Documents/ia/controllers/cnn/checkpoints/checkpoint_fold_3.pth
Resumed from epoch 40. Best validation loss so far: 0.0030

--- Starting Model Training for Fold 3 from Epoch 41 ---

--- Starting Fold 4/5 ---
Tamanho da característica da câmera achatada calculado dinamicamente: 24576
Resuming training from checkpoint: /home/dino/Documents/ia/controllers/cnn/checkpoints/checkpoint_fold_4.pth
Resumed from epoch 36. Best validation loss so far: 0.0029

--- Starting Model Training for Fold 4 from Epoch 37 ---


Fold 4/5 Epoch 37/40 - Loss: 0.0035, MAE: 0.0395 - Val Loss: 0.0032, Val MAE: 0.0329


Fold 4/5 Epoch 38/40 - Loss: 0.0034, MAE: 0.0383 - Val Loss: 0.0030, Val MAE: 0.0309


Fold 4/5 Epoch 39/40 - Loss: 0.0034, MAE: 0.0385 - Val Loss: 0.0030, Val MAE: 0.0308


Fold 4/5 Epoch 40/40 - Loss: 0.0032, MAE: 0.0375 - Val Loss: 0.0029, Val MAE: 0.0282
  -> New best validation loss: 0.0029. Saving checkpoint...

--- Starting Fold 5/5 ---
Tamanho da característica da câmera achatada calculado dinamicamente: 24576
Resuming training from checkpoint: /home/dino/Documents/ia/controllers/cnn/checkpoints/checkpoint_fold_5.pth
Resumed from epoch 39. Best validation loss so far: 0.0022

--- Starting Model Training for Fold 5 from Epoch 40 ---


Fold 5/5 Epoch 40/40 - Loss: 0.0029, MAE: 0.0351 - Val Loss: 0.0021, Val MAE: 0.0254
  -> New best validation loss: 0.0021. Saving checkpoint...
Training plot saved to '/home/dino/Documents/ia/controllers/cnn/training_history.png'

Training complete.


In [29]:
def fine_tune_model(best_checkpoint_path: Path, full_dataset_path: Path, fine_tune_epochs: int = 10, fine_tune_lr: float = 1e-5):
    print(f"\n--- Starting Fine-tuning on Entire Dataset ---")
    print(f"Loading best model from: {best_checkpoint_path}")

    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the full dataset
    (X_cam, X_lidar), y = load_data(full_dataset_path)
    if X_cam is None or X_cam.shape[0] == 0:
        print("Exiting fine-tuning: No data loaded for full dataset.")
        return

    print(
        f"Full dataset loaded. Shapes: Cam={X_cam.shape}, Lidar={X_lidar.shape}, Target={y.shape}")

    # Create DataLoader for the full dataset
    full_dataset = TensorDataset(torch.tensor(
        X_cam), torch.tensor(X_lidar), torch.tensor(y))
    full_loader = DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=True)

    # Instantiate model and load state dict
    model = CNNNavigationModel(lidar_shape_in=X_lidar.shape[1]).to(device)
    checkpoint = torch.load(best_checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

    # Optimizer for fine-tuning (can be a new one with lower LR)
    optimizer = optim.Adam(model.parameters(), lr=fine_tune_lr)
    # Optionally load optimizer state if resuming fine-tuning
    if 'optimizer_state_dict' in checkpoint:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    criterion = nn.MSELoss()
    mae_criterion = nn.L1Loss()

    print(f"Fine-tuning for {fine_tune_epochs} epochs with LR: {fine_tune_lr}")

    model.train()  # Set model to training mode
    for epoch in range(fine_tune_epochs):
        running_loss, running_mae = 0.0, 0.0
        progress_bar = tqdm(
            full_loader, desc=f"Fine-tune Epoch {epoch+1}/{fine_tune_epochs}", leave=False)
        for cam_batch, lidar_batch, target_batch in progress_bar:
            cam_batch, lidar_batch, target_batch = cam_batch.to(
                device), lidar_batch.to(device), target_batch.to(device)
            optimizer.zero_grad()
            outputs = model(cam_batch, lidar_batch)
            loss = criterion(outputs, target_batch)
            mae = mae_criterion(outputs, target_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_mae += mae.item()
            progress_bar.set_postfix(loss=loss.item(), mae=mae.item())

        epoch_loss = running_loss / len(full_loader)
        epoch_mae = running_mae / len(full_loader)
        print(
            f"Fine-tune Epoch {epoch+1}/{fine_tune_epochs} - Loss: {epoch_loss:.4f}, MAE: {epoch_mae:.4f}")

    # Save the final fine-tuned model
    final_model_path = SCRIPT_DIR / 'final_cnn_model.pth'
    torch.save(model.state_dict(), final_model_path)
    print(f"Fine-tuned model saved to: {final_model_path}")

if best_overall_checkpoint_path:
    fine_tune_model(best_overall_checkpoint_path, DATASET_PATH)
else:
    print("\nNo best model found for fine-tuning.")


--- Starting Fine-tuning on Entire Dataset ---
Loading best model from: /home/dino/Documents/ia/controllers/cnn/checkpoints/checkpoint_fold_5.pth
Loading samples from '/home/dino/Documents/ia/controllers/cnn/cnn_dataset.h5'...
Full dataset loaded. Shapes: Cam=(7493, 4, 40, 200), Lidar=(7493, 20), Target=(7493, 2)
Tamanho da característica da câmera achatada calculado dinamicamente: 24576
Fine-tuning for 10 epochs with LR: 1e-05


Fine-tune Epoch 1/10 - Loss: 0.0028, MAE: 0.0341


Fine-tune Epoch 2/10 - Loss: 0.0027, MAE: 0.0334


Fine-tune Epoch 3/10 - Loss: 0.0026, MAE: 0.0325


Fine-tune Epoch 4/10 - Loss: 0.0026, MAE: 0.0324


...

Fine-tune Epoch 6/10 - Loss: 0.0025, MAE: 0.0314


Fine-tune Epoch 7/10 - Loss: 0.0024, MAE: 0.0308


Fine-tune Epoch 8/10 - Loss: 0.0023, MAE: 0.0300


Fine-tune Epoch 9/10 - Loss: 0.0023, MAE: 0.0297


Fine-tune Epoch 10/10 - Loss: 0.0022, MAE: 0.0292
Fine-tuned model saved to: /home/dino/Documents/ia/controllers/cnn/final_cnn_model.pth


In [31]:
# --- Evaluate on Test Set ---
print("\n--- Evaluating on Test Set ---")

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the test data if not already loaded
if 'X_cam_test' not in locals() or 'X_lidar_test' not in locals() or 'y_test' not in locals():
    print("Loading test data...")
    (X_cam_test, X_lidar_test), y_test = load_data(Path('/home/dino/Documents/ia/controllers/cnn/test.h5'))

# Create DataLoader for the test set
test_dataset = TensorDataset(torch.tensor(X_cam_test), torch.tensor(X_lidar_test), torch.tensor(y_test))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Instantiate the model
model = CNNNavigationModel(lidar_shape_in=X_lidar_test.shape[1]).to(device)

# Load the final fine-tuned model state
final_model_path = SCRIPT_DIR / 'final_cnn_model.pth'
if final_model_path.exists():
    print(f"Loading final model from: {final_model_path}")
    model.load_state_dict(torch.load(final_model_path, map_location=device))
else:
    print("Error: Final model 'final_cnn_model.pth' not found!")

# Evaluate the model
model.eval()
test_loss = 0.0
test_mae = 0.0
criterion = nn.MSELoss()
mae_criterion = nn.L1Loss()

with torch.no_grad():
    for cam_batch, lidar_batch, target_batch in test_loader:
        cam_batch, lidar_batch, target_batch = cam_batch.to(device), lidar_batch.to(device), target_batch.to(device)
        outputs = model(cam_batch, lidar_batch)
        test_loss += criterion(outputs, target_batch).item()
        test_mae += mae_criterion(outputs, target_batch).item()

avg_test_loss = test_loss / len(test_loader)
avg_test_mae = test_mae / len(test_loader)

print(f"\nFinal Test Results:")
print(f"  - Test Loss (MSE): {avg_test_loss:.4f}")
print(f"  - Test MAE: {avg_test_mae:.4f}")


--- Evaluating on Test Set ---
Tamanho da característica da câmera achatada calculado dinamicamente: 24576
Loading final model from: /home/dino/Documents/ia/controllers/cnn/final_cnn_model.pth

Final Test Results:
  - Test Loss (MSE): 0.0030
  - Test MAE: 0.0293
